# Read-Only Properties

- To create a read-only property, we just need to create a property with only the get
accessor defined
    - not truly read-only since underlying storage variable could be accessed directly
- useful for computed properties

```python
import math
class Circle:
    def __init__(self, r):
        self.r = r
    def area(self):
        return math.pi * self.r * self.r

c = Circle(1) 
c.area()    # returns 3.14...

# we can use decorator instead to make read-only and use the area as a property of a circle
# not as a function
# feels more natural since area is really a property of a circle

class Circle:
    def __init__(self, r):
        self.r = r
    @property
    def area(self):
        return math.pi * self.r * self.r

c = Circle(1)
# not here area is not a property
c.area  # returns 3.14..

```
- `Application: Caching Computed Properties`
    - Using propoerty setters is sometimes useful for controlling how other computed properties are cached
    - `Circle`
        - `area` is computed properly
            - `lazy` computation - only calculate `area` if requested
            - cache value - so if re-requested we save the computation
                - but what if someone changes the `radius` ?
                - need to `invalidate` the cache
        - control setting the `radius` using a `property`
            - we are now `aware` when the property has been changed
            
```python
import math

class Circle:
    def __init__(self, r):
        self._r = r
        self._area = None   # seting _area cache to None

    @property
    def radius(self):
        return self._r

    @radius.setter
    def radius(self, r):
        if r < 0:
            raise ValueError('Radius must be non-negative')
        self._r = r
        self._area = None   # invalidate cache
    
    # read-only property
    @property
    def area(self):
        if self._area is None:
            self._area = math.pi * (self.radius ** 2)
        return self._area

```                            

In [5]:
from math import pi

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        print('calculatin area...')
        return pi * (self.radius ** 2)



In [6]:
c = Circle(1)

In [7]:
c.radius

1

In [8]:
c.area

calculatin area...


3.141592653589793

In [9]:
# so every time I reinvoke the area its going to recalculate it, thats not very good
# thats why we use cache
# so we only wanna recalculate it if the radius is changed, but if the radius is not 
# changed then theres no point calculating the area again and again

# So how we are going to know if the are is changed
# we are going to use a property to keep track when the raidus is being set
# and then we are going to cache the area itself
# if the radius hasnt change we will just pull the value out of the cache
# if the radius has change we will recalculate the area put it in the cache and then use that
c.area

calculatin area...


3.141592653589793

In [10]:
class Circle:
    def __init__(self, radius):
        self._radius = radius # pesudo private variable
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        self._area = None
        self._radius = value
    
    @property
    def area(self):
        if self._area is None: # means i havent calculated the area
            print('Calculating the area')
            self._area = pi * (self.radius ** 2)
        return self._area



In [11]:
c = Circle(1)

In [12]:
c.area

Calculating the area


3.141592653589793

In [14]:
# observe it didnt calculate the area again it just returned the cached area(value)
c.area


3.141592653589793

In [15]:
c.radius = 2

In [17]:
# area is set to None by setter
c.__dict__

{'_radius': 2, '_area': None}

In [18]:
c.area

Calculating the area


12.566370614359172

In [19]:
c.area

12.566370614359172

In [20]:
import urllib
from time import perf_counter

In [24]:
class WebPage:
    def __init__(self, url):
        self.url = url
        self._page = None
        self._load_time_secs = None
        self._page_size = None
    
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        self._page = None
        
    @property
    def page(self):
        if self._page is None:
            self.download_page()
        return self._page
    
    @property
    def page_size(self):
        if self._page is None:
            self.download_page()
        return self._page_size
    
    @property
    def time_elapsed(self):
        if self._page is None:
            self.download_page()
        return self._load_time_secs
    
    def download_page(self):
        self._page_size = None
        self._load_time_secs = None
        start_time = perf_counter()
        with urllib.request.urlopen(self.url) as f:
            self._page = f.read()
        end_time = perf_counter()
        self._page_size = len(self._page)
        self._load_time_secs = end_time - start_time

In [25]:
urls = [
    'https://www.google.com',
    'https://www.python.org',
    'https://www.yahoo.com'
]

In [26]:
for url in urls:
    page = WebPage(url)
    print(f'{url}\tsize={format(page.page_size,"_" )}\telapsed={page.time_elapsed:.2f} secs')

# so this is a example of using properties, and calculator properties and doign lazy loading
# and caching and stuff like that.

# hopefully you get an understanding of when you might or might not want to use a bare attribute
# so very often when we write classes in Python we are gonna use bare attributes until such
# a time when we need them to be properties then we can transform them into properties at that
# time without braking backward compatibility

https://www.google.com	size=15_111	elapsed=1.05 secs
https://www.python.org	size=48_981	elapsed=0.40 secs
https://www.yahoo.com	size=452_125	elapsed=1.96 secs
